# 0. 初めに
今回はKaggleのcompetition**New York City Taxi Fare Prediction**に参加してみました。
回帰予測のチュートリアル感覚で、データ数が多いこと以外は初心者でもサクサクできたと思います。
    https://www.kaggle.com/c/new-york-city-taxi-fare-prediction

今回は先輩方や同期の意見を参考にしたり、初心者なりにまとめたつもりなので温かい目で見ていってくだされば幸いです。
かつフィードバックもいただければと期待しております。
ではよろしくお願いします。




# 1. 前処理

In [ ]:
#その他もろもろをインストール
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly
%matplotlib inline

import plotly.offline as offline
import plotly.graph_objs as go
offline.init_notebook_mode()

!pip install lightgbm
import datetime as dt
import lightgbm as lgbm

import sklearn
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

import io
import os
import gc

In [ ]:
#データを読み込む
train_data = pd.read_csv('../input/train.csv', nrows=5000000)

#プロット用に作っておく
train_data5k = pd.read_csv('../input/train.csv', nrows=5000)

訓練データ量がかなり多いので, nrowsで読み込む量を調節する.  
ただ過学習を防ぐためにも訓練データ量を増やしたほうが良いので、学習モデルが決まればPCが許す限り増やしてみるべき.   
(私はこんな感じで増やしていきました。　5000⇒10000⇒・・・⇒1000000)

In [ ]:
#データの中身
train_data.head()

今回のコンペティションの目的は, データからお客さんの支払ったタクシーの運賃を予測するといった内容.   
まずはざっくりと与えられている情報の意味を確認した.   
### fare_amount:タクシーの運賃
### pickup_datetime:乗客を乗せた時間
### pickup_longitude, picup_latitude: 乗客を乗せた経度, 緯度
### dropoff_longitude, dropoff_latitude: 乗客をおろした経度, 緯度
### passenger_count: 乗せた乗客数


加えて, タクシー運賃に関係のある情報をメモしておいた.

- 初乗り運賃は2.5ドル
- 一般料金 : 1/5マイル(320m)で0.5ドル
- キーワードは空港   
    * ラガーディア空港: 一般
        40.6927796,-74.1838495
    * JFK空港：５２＋Toll + 4.5(16~20時)
       40.6448435,-73.7818262
    * JFK空港とマンハッタン以外: 一般料金
    * ニューアーク空港: 一般+ Toll 
        40.6876721,-74.177831
- 2012.09より17%値上げ

In [ ]:
#データのタイプも見ておく
train_data.info()

ここで一応作っておいたcolumns単位でNaN数の割合を出す関数を使ってみる.   
拾い物だが, これがあると欠損値の数が多すぎるデータカラムを最初からdropすることもできるので今のところ便利である.   

In [ ]:
#欠損値をカウント
def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() /len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis = 1)
    mis_val_table_ren_columns = mis_val_table.rename(columns={0:'Missing Values', 1:'% of Total Values'})
    #欠損値の割合
    mis_val_table_ren_columns=mis_val_table_ren_columns[mis_val_table_ren_columns.iloc[:,1]!=
                                                        0].sort_values('% of Total Values',ascending=False).round(1)
    #カラム数を表示
    print("You selected dataframe has" + str(df.shape[1]) + "columns.\n"
         "There are" + str(mis_val_table_ren_columns.shape[0]) + 
         "columns that have missing values.")
    
    #dataframe中の欠損値の情報
    return mis_val_table_ren_columns

In [ ]:
missing = missing_values_table(train_data)
missing

In [ ]:
#376個ずつある. 全体の行数に対しては0%の割合なので影響はほぼないと考えられる. #行単位(axis=0)で消しておく
train_data = train_data.dropna(how='any', axis=0)

#NaN数がなくなったことを確認
missing = missing_values_table(train_data)
missing

推理した情報を頼りに前処理にて意味のありそうな訓練データに加工する.    
とりあえず, データタイプがobject(文字)のものは扱いずらいので加工しておく.

常識的に考えて乗客数に運賃は関係なさそうなので, 「passenger_count」はdrop出来そう.

In [ ]:
#datetimeを分離する
train_data['pickup_datetime'] = pd.to_datetime(train_data['pickup_datetime'])

train_data['pickup_datetime_month'] = train_data['pickup_datetime'].dt.month
train_data['pickup_datetime_year'] = train_data['pickup_datetime'].dt.year
train_data['pickup_datetime_day_of_week'] = train_data['pickup_datetime'].dt.weekday
train_data['pickup_datetime_day_of_hour'] = train_data['pickup_datetime'].dt.hour

In [ ]:
correlation = np.corrcoef(train_data['fare_amount'], train_data['passenger_count'])

#乗客数の影響はとても小さいので消す
train_data = train_data.drop(['passenger_count'], axis = 1)

#ついでに[key]の情報もdatetimeに含まれているので消す
train_data = train_data.drop(['pickup_datetime'], axis =1)
train_data = train_data.drop(['key'], axis = 1)

train_data.head()

latitude, longitudeは座標情報でもあるため, ライブラリのplotlyでNY周辺の地図上にプロットしてみる.   
黄色の点が乗車位置で, 青色の点が降車位置である.   


ここからわかることとしては明らかにタクシーで行くことが不可能な場所に点が存在する.   
これらは外れ値として処理しておく.

In [ ]:
data = [go.Scattermapbox(
            lat= train_data5k['pickup_latitude'] ,
            lon= train_data5k['pickup_longitude'],
            mode='markers',
            marker=dict(
                size= 4,
                color = 'gold',
                opacity = .8,
            ),
          )]
layout = go.Layout(autosize=False,
                   mapbox= dict(accesstoken="pk.eyJ1Ijoic2hhejEzIiwiYSI6ImNqYXA3NjhmeDR4d3Iyd2w5M2phM3E2djQifQ.yyxsAzT94VGYYEEOhxy87w",
                                bearing=10,
                                pitch=60,
                                zoom=13,
                                center= dict(
                                         lat=40.721319,
                                         lon=-73.987130),
                                style= "mapbox://styles/shaz13/cjiog1iqa1vkd2soeu5eocy4i"),
                    width=900,
                    height=600, title = "Pick up Locations in NewYork")

fig = dict(data=data, layout=layout)
offline.iplot(fig)

In [ ]:
 data = [go.Scattermapbox(
            lat= train_data5k['dropoff_latitude'] ,
            lon= train_data5k['dropoff_longitude'],
            mode='markers',
            marker=dict(
                size= 4,
                color = 'cyan',
                opacity = .8,
            ),
          )]
layout = go.Layout(autosize=False,
                   mapbox= dict(accesstoken="pk.eyJ1Ijoic2hhejEzIiwiYSI6ImNqYXA3NjhmeDR4d3Iyd2w5M2phM3E2djQifQ.yyxsAzT94VGYYEEOhxy87w",
                                bearing=10,
                                pitch=60,
                                zoom=13,
                                center= dict(
                                         lat=40.721319,
                                         lon=-73.987130),
                                style= "mapbox://styles/shaz13/cjk4wlc1s02bm2smsqd7qtjhs"),
                    width=900,
                    height=600, title = "Drop off locations in Newyork")
fig = dict(data=data, layout=layout)
offline.iplot(fig)

In [ ]:
#longitude-75~-72, latitude39~43
train_data = train_data[(train_data['pickup_longitude'] > -75 )& (train_data['pickup_longitude'] < -72)]
train_data = train_data[(train_data['dropoff_longitude'] > -75 )& (train_data['dropoff_longitude'] < -72)]
train_data = train_data[(train_data['pickup_latitude'] > 39 )& (train_data['pickup_latitude'] < 43)]
train_data = train_data[(train_data['dropoff_latitude'] > 39 )& (train_data['dropoff_latitude'] < 43)]

また経度緯度は地球上の座標情報であるため, 乗車位置・降車位置を決定することで移動距離が得られる.   
タクシーは移動距離により決定するため移動距離を計算しておくことは大変重大であると考えられる.   
さらに前述したように, 空港から乗ったり下りたりする時の運賃は一般料金と別になることがある. 
そのことを考慮するために一部のランドマークの位置情報を加えておく.   
今回は乗車位置～ランドマークの位置の距離とランドマークの位置～降車位置の和を考慮した.  
すなわちあるランドマークXXXから乗った(または降りた)人は, 「XXX_coord」の値が小さくなる.  

In [ ]:
#landmark
def add_airport_dist(dataset):
    """
    Return minumum distance from pickup or dropoff coordinates to each airport.
    JFK: John F. Kennedy International Airport
    EWR: Newark Liberty International Airport
    LGA: LaGuardia Airport
    SOL: Statue of Liberty 
    NYC: Newyork Central
    """
    jfk_coord = (40.639722, -73.778889)
    ewr_coord = (40.6925, -74.168611)
    lga_coord = (40.77725, -73.872611)
    sol_coord = (40.6892,-74.0445) # Statue of Liberty
    nyc_coord = (40.7141667,-74.0063889) 
    
    
    pickup_lat = dataset['pickup_latitude']
    dropoff_lat = dataset['dropoff_latitude']
    pickup_lon = dataset['pickup_longitude']
    dropoff_lon = dataset['dropoff_longitude']
    
    pickup_jfk = sphere_dist(pickup_lat, pickup_lon, jfk_coord[0], jfk_coord[1]) 
    dropoff_jfk = sphere_dist(jfk_coord[0], jfk_coord[1], dropoff_lat, dropoff_lon) 
    pickup_ewr = sphere_dist(pickup_lat, pickup_lon, ewr_coord[0], ewr_coord[1])
    dropoff_ewr = sphere_dist(ewr_coord[0], ewr_coord[1], dropoff_lat, dropoff_lon) 
    pickup_lga = sphere_dist(pickup_lat, pickup_lon, lga_coord[0], lga_coord[1]) 
    dropoff_lga = sphere_dist(lga_coord[0], lga_coord[1], dropoff_lat, dropoff_lon)
    pickup_sol = sphere_dist(pickup_lat, pickup_lon, sol_coord[0], sol_coord[1]) 
    dropoff_sol = sphere_dist(sol_coord[0], sol_coord[1], dropoff_lat, dropoff_lon)
    pickup_nyc = sphere_dist(pickup_lat, pickup_lon, nyc_coord[0], nyc_coord[1]) 
    dropoff_nyc = sphere_dist(nyc_coord[0], nyc_coord[1], dropoff_lat, dropoff_lon)
    
    
    
    dataset['jfk_dist'] = pickup_jfk + dropoff_jfk
    dataset['ewr_dist'] = pickup_ewr + dropoff_ewr
    dataset['lga_dist'] = pickup_lga + dropoff_lga
    dataset['sol_dist'] = pickup_sol + dropoff_sol
    dataset['nyc_dist'] = pickup_nyc + dropoff_nyc
    
    return dataset

def sphere_dist(pickup_lat, pickup_lon, dropoff_lat, dropoff_lon):
    """
    Return distance along great radius between pickup and dropoff coordinates.
    """
    #Define earth radius (km)
    R_earth = 6371
    #Convert degrees to radians
    pickup_lat, pickup_lon, dropoff_lat, dropoff_lon = map(np.radians,
                                                             [pickup_lat, pickup_lon, 
                                                              dropoff_lat, dropoff_lon])
    #Compute distances along lat, lon dimensions
    dlat = dropoff_lat - pickup_lat
    dlon = dropoff_lon - pickup_lon
    
    #Compute haversine distance
    a = np.sin(dlat/2.0)**2 + np.cos(pickup_lat) * np.cos(dropoff_lat) * np.sin(dlon/2.0)**2
    return 2 * R_earth * np.arcsin(np.sqrt(a))

In [ ]:
train_data['distance[km]'] = [sphere_dist(train_data.iloc[i,1], train_data.iloc[i,2], train_data.iloc[i,3], train_data.iloc[i,4]) for i in range(0, len(train_data['dropoff_latitude']))]
add_airport_dist(train_data)

In [ ]:
#移動距離に対する運賃を可視化しておく
plt.scatter(train_data['distance[km]'], train_data['fare_amount'])

縦軸は運賃であるが, 負の値をとるものがあるためこれも外しておく.    
また, 移動距離が小さすぎるものも取り除いておく.

In [ ]:
train_data = train_data[(train_data['fare_amount'] > 0.01)]
train_data = train_data[(train_data['distance[km]'] > 0.01)]

In [ ]:
print(train_data.shape)
#plt.xlim(0,10)
plt.scatter(train_data['distance[km]'], train_data['fare_amount'])

In [ ]:
train_data.head(15)

In [ ]:
#値上げしたことがよく分かった
sns.barplot('pickup_datetime_month', 'fare_amount' ,hue ='pickup_datetime_year', data=train_data)

In [ ]:
train_data['raised'] = 0
train_data.loc[train_data['pickup_datetime_year'] > 2012, 'raised'] = 1
train_data.loc[(train_data['pickup_datetime_month'] >8) & (train_data['pickup_datetime_year'] == 2012), 'raised'] = 1

In [ ]:
#曜日の依存はない？
sns.barplot('pickup_datetime_day_of_week', 'fare_amount', hue = 'pickup_datetime_month', data=train_data)

In [ ]:
#朝は遠くから来る人が多い
sns.barplot('pickup_datetime_day_of_hour', 'distance[km]', data=train_data)

日時の情報に関しては重要度は**曜日<年<時間=月**の順に低いと予想した   
一応相関係数も見ておく

In [ ]:
correlation_week = np.corrcoef(train_data['fare_amount'], train_data['pickup_datetime_day_of_week'])
correlation_year= np.corrcoef(train_data['fare_amount'], train_data['pickup_datetime_year'])
correlation_month = np.corrcoef(train_data['fare_amount'], train_data['pickup_datetime_month'])
correlation_hour= np.corrcoef(train_data['fare_amount'], train_data['pickup_datetime_day_of_hour'])

print("曜日との相関係数は:",correlation_week[0,1])
print("年との相関係数は:",correlation_year[0,1])
print("月との相関係数は:",correlation_month[0,1])
print("時間との相関係数は:",correlation_hour[0,1])

In [ ]:
sns.pairplot(train_data,
             x_vars=['pickup_datetime_month', 'pickup_datetime_year','pickup_datetime_day_of_week',
                     'pickup_datetime_day_of_hour','jfk_dist','ewr_dist','lga_dist','sol_dist','distance[km]'],
             y_vars=['fare_amount'])

In [ ]:
train_data.head()

# 2. 学習
リーダーボードでも強かったlight勾配ブースティングで学習を行った.   
また, データのばらつきが正規分布っぽくはなかったので正規化ではなく標準化も行った.    
ハイパーパラメータはかなり大雑把に決めたものである. その中でも時間が許す限りはlgbにおける学習率を下げたい.   

In [ ]:
y = train_data['fare_amount']
X = train_data.drop(['fare_amount'], axis=1)

In [ ]:
#MinMaxScalerで標準化してみる
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_scaler = scaler.fit(X)
print("max:{}, min:{}".format(scaler.data_max_, scaler.data_min_))
X_mms= scaler.transform(X)

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X_mms,y,random_state=42,test_size=0.30)

gc.collect()

params = {
        'boosting_type':'gbdt',
        'objective': 'regression',
        'nthread': 4,
        'num_leaves': 31,
        'learning_rate': 0.03,
        'max_depth': -1,
        'subsample': 0.8,
        'bagging_fraction' : 1,
        'max_bin' : 5000 ,
        'bagging_freq': 20,
        'colsample_bytree': 0.6,
        'metric': 'rmse',
        'min_split_gain': 0.5,
        'min_child_weight': 1,
        'min_child_samples': 10,
        'scale_pos_weight':1,
        'zero_as_missing': True,
        'seed':0,
        'num_rounds':50000
    }
    
train_set = lgbm.Dataset(x_train, y_train, silent=False)
valid_set = lgbm.Dataset(x_test, y_test, silent=False)
model = lgbm.train(params, train_set = train_set, num_boost_round=500,early_stopping_rounds=500,verbose_eval=500, valid_sets=valid_set)
gc.collect()

特徴量の重要度も調べてみた.   
「raised」の値が小さいが, 先ほどのプロットにより有意そうなので消さないでおく.

In [ ]:
lgbm.plot_importance(model, figsize=(12, 10))
train_data.head(1)

# 3.提出
とりあえず行った処理をテストデータのほうにも一通り行う.

In [ ]:
#データを読み込む
test_data = pd.read_csv('../input/test.csv')
print(test_data.info())
test_data.head()

In [ ]:
missing = missing_values_table(test_data)
missing

欠損値はないのでとりあえず一安心.

In [ ]:
#datetimeを分離する
test_data['pickup_datetime'] = pd.to_datetime(test_data['pickup_datetime'])

test_data['pickup_datetime_month'] = test_data['pickup_datetime'].dt.month
test_data['pickup_datetime_year'] = test_data['pickup_datetime'].dt.year
test_data['pickup_datetime_day_of_week'] = test_data['pickup_datetime'].dt.weekday
test_data['pickup_datetime_day_of_hour'] = test_data['pickup_datetime'].dt.hour

#乗客数の影響はとても小さいので消す
test_data = test_data.drop(['passenger_count'], axis = 1)

#ついでに[key]の情報もdatetimeに含まれているので消す
test_data = test_data.drop(['pickup_datetime'], axis =1)
test_data = test_data.drop(['key'], axis = 1)

test_data['distance[km]'] = [sphere_dist(test_data.iloc[i,0], test_data.iloc[i,1], test_data.iloc[i,2], test_data.iloc[i,3]) for i in range(0, len(test_data['dropoff_latitude']))]
add_airport_dist(test_data)

#値上げ前考慮
test_data['raised'] = 0
test_data.loc[test_data['pickup_datetime_year'] > 2012, 'raised'] = 1
test_data.loc[(test_data['pickup_datetime_month'] >8) & (test_data['pickup_datetime_year'] == 2012), 'raised'] = 1

In [ ]:
test_data.head()

タクシーでいけない範囲に座標があるか調べておく.

In [ ]:
#大体NYC内にいることがわかる
sns.pairplot(x_vars=['pickup_latitude','dropoff_latitude'], y_vars=['pickup_longitude', 'dropoff_longitude'], data = test_data)

In [ ]:
#train_dataとcolumnの順が同じになるようにしておく
test_data = test_data.ix[:,['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','pickup_datetime_month','pickup_datetime_year','pickup_datetime_day_of_week','pickup_datetime_day_of_hour','distance[km]','jfk_dist','ewr_dist','lga_dist','sol_dist','nyc_dist','raised']]

In [ ]:
#標準化しておく
test_data = scaler.transform(test_data)

In [ ]:
#予測
submission = model.predict(test_data)
taxi_data = pd.read_csv('../input/test.csv')
output = pd.DataFrame({'key':taxi_data.key, 'fare_amount':submission})
output.to_csv('taxi_prediction.csv', index=False)